In [1]:
import os
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate:float
    params_patience:int

In [3]:

from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-04-12 11:41:42.387724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 11:41:42.901497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_patience=params.PATIENCE
        )

        return training_config


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def compile_model(self):
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                        verbose=1, 
                                                        patience=self.config.params_patience,
                                                        restore_best_weights=True)]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.compile_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-04-12 12:27:41,990: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-12 12:27:41,991: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-12 12:27:41,992: INFO: common: created directory at: artifacts]
[2024-04-12 12:27:41,992: INFO: common: created directory at: artifacts/training]
[2024-04-12 12:27:42,087: WARNING: legacy_h5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/100


/home/jamie/anaconda3/envs/chest_xray/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - accuracy: 0.5579 - loss: 10.6828 - val_accuracy: 0.5735 - val_loss: 32.6360
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.5223 - loss: 18.7211 - val_accuracy: 0.8382 - val_loss: 1.2220
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.5645 - loss: 9.6027 - val_accuracy: 0.5735 - val_loss: 10.7406
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.7110 - loss: 5.4451 - val_accuracy: 0.9853 - val_loss: 0.0568
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.7048 - loss: 4.6438 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.8315 - loss: 2.2884 - val_accuracy: 0.9853 - val_loss: 0.0162
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - accuracy: 0.7926 - loss: 2.8255 - val_accuracy: 0.4265 - val_loss: 13.7232
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.7695 - loss: 3.6387 - val_accuracy: 0.4